In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import openai
from dotenv import load_dotenv, find_dotenv
import os
import sys

sys.path.append("../..")


_ = load_dotenv(find_dotenv())  # read local .env file

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
import langchain

langchain.debug = False

In [4]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings, OpenAIEmbeddings

embeddings_hf = HuggingFaceInferenceAPIEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    api_key=HUGGINGFACEHUB_API_TOKEN,
)

embeddings_openai = OpenAIEmbeddings()

/home/codespace/.python/current/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
import asyncio
from pyppeteer import launch
from bs4 import BeautifulSoup
import nest_asyncio
page = None
async def main(url):
    global page
    browser = await launch()
    page = await browser.newPage()
    await page.goto(url)
    
    # Get the page content after JavaScript execution
    page_content = await page.content()

    # Close the browser
    await browser.close()

    # Now you can use BeautifulSoup to parse the page content
    soup = BeautifulSoup(page_content, 'html.parser')

    # Perform scraping using BeautifulSoup on the dynamically rendered content
    # ...return soup
    # print(soup.prettify())
    return soup

nest_asyncio.apply()
url = "https://www.mdcsoftware.com.vn/"
soup = asyncio.get_event_loop().run_until_complete(main(url))



In [7]:
recruiment = asyncio.get_event_loop().run_until_complete(main(url+"recruitment"))

In [20]:
from langchain.text_splitter import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
    ("h5", "Header 5"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

html_header_splits = html_splitter.split_text(soup.text[:12] + soup.text[58:] + "\n" + recruiment.text[113:-225])

chunk_size = 200
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

docs = text_splitter.split_documents(html_header_splits)

In [ ]:
from agentic_chunking import get_propositions

propositions = get_propositions(soup.text[:12] + soup.text[58:] + "\n" + recruiment.text[113:-225])

In [10]:
propositions

['About Us: MDC Software is a technology company based in Hanoi City, specializing in mobile development and web design for individuals, companies, and startups.',
 'Services and Products: MDC Software offers a range of services including web application development, artificial intelligence, internet of things, UX design, streaming solutions, product marketing, payment integration, blockchain, technical advice, and more. They also have a portfolio of products such as iVPN, iMatch, Advanced IPTV Player, and experience in augmented reality.',
 'Recruitment: MDC Software is currently recruiting front-end react native developers and freshers for new projects. The job description includes JavaScript programming, participation in project development, and application development on platforms like React Native and ReactJS.',
 'Address and Contact: MDC VIETNAM MEDIA AND TECHNOLOGY COMPANY LIMITED is located on the 15th Floor, Viet A Building, No. 9 Duy Tan, Cau Giay, Hanoi. They can be contacte

In [13]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma, FAISS

db = await FAISS.afrom_texts(propositions,embeddings_openai)

In [14]:
from langchain.chat_models import ChatOpenAI

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableParallel,
)
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 2})

template = """You're helpful assistant, please answer the question based only on the following context:
{context}
Your response based on user's question language.
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0,
    callbacks=[StreamingStdOutCallbackHandler()],
    streaming=True,
)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [19]:
chain.invoke("MDC là gì?")

MDC là một công ty công nghệ có trụ sở tại thành phố Hà Nội, chuyên về phát triển ứng dụng di động và thiết kế web cho cá nhân, công ty và các doanh nghiệp khởi nghiệp. Địa chỉ của MDC VIETNAM MEDIA AND TECHNOLOGY COMPANY LIMITED là tầng 15, tòa nhà Viet A, số 9 Đuy Tân, Cầu Giấy, Hà Nội. Để liên hệ, bạn có thể gọi theo số 0868.733.900 hoặc gửi email đến hello@mdcsoftware.com.vn.

'MDC là một công ty công nghệ có trụ sở tại thành phố Hà Nội, chuyên về phát triển ứng dụng di động và thiết kế web cho cá nhân, công ty và các doanh nghiệp khởi nghiệp. Địa chỉ của MDC VIETNAM MEDIA AND TECHNOLOGY COMPANY LIMITED là tầng 15, tòa nhà Viet A, số 9 Đuy Tân, Cầu Giấy, Hà Nội. Để liên hệ, bạn có thể gọi theo số 0868.733.900 hoặc gửi email đến hello@mdcsoftware.com.vn.'